In [1]:
import requests
from bs4 import BeautifulSoup
import pandas as pd
import numpy as np
import re

In [2]:
url = 'https://www.basketball-reference.com/draft/NBA_1997.html'

In [3]:
req0 = requests.get(url=url)
soup = BeautifulSoup(req0.content, 'lxml')

data_ = soup.find_all('tr')[1:]

player_data = pd.read_html(str(soup.select('table')), match='Player')[0]
player_data.columns = player_data.columns.droplevel(0)

player_data = player_data.loc[(player_data['Player'] != 'Player') & 
                              (player_data['Yrs'] != 'Totals')].reset_index(drop=True)

In [4]:
playerid = []
for j in data_:
    if len(j.find_all('a')) == 4:
        pid_v0 = j.find_all('a')[2].get('href')
        pid_v1 = pid_v0.replace('players/', '').replace('.html', '').split('/')[2]
        playerid.append(pid_v1)
    elif len(j.find_all('a')) == 3:
        pid_v0 = j.find_all('a')[2].get('href')
        if 'players' in pid_v0:
            pid_v1 = pid_v0.replace('players/', '').replace('.html', '').split('/')[2]
            playerid.append(pid_v1)
        else:
            playerid.append('no_playerid')
    elif len(j.find_all('a')) == 2:
        playerid.append('no_playerid')


In [6]:
player_data.iloc[:, 1:].head()

,Pk,Tm,Player,College,Yrs,G,MP,PTS,TRB,AST,...,3P%,FT%,MP,PTS,TRB,AST,WS,WS/48,BPM,VORP
0,1,SAS,Tim Duncan,Wake Forest University,19,1392,47368,26496,15091,4225,...,.179,.696,34.0,19.0,10.8,3.0,206.4,.209,5.5,89.3
1,2,PHI,Keith Van Horn,University of Utah,9,575,18150,9206,3909,900,...,.361,.835,31.6,16.0,6.8,1.6,41.7,.110,-0.2,8.4
2,3,BOS,Chauncey Billups,University of Colorado,17,1043,33008,15802,2992,5636,...,.387,.894,31.6,15.2,2.9,5.4,120.8,.176,2.5,37.6
3,4,VAN,Antonio Daniels,Bowling Green State University,13,872,19671,6623,1613,2934,...,.311,.793,22.6,7.6,1.8,3.4,47.0,.115,-1.2,4.1
4,5,DEN,Tony Battie,Texas Tech University,14,837,17649,5082,4287,522,...,.162,.690,21.1,6.1,5.1,0.6,36.4,.099,-1.6,1.9


In [ ]:
len(playerid), len(player_data)

In [ ]:
'players' in data_[55].find_all('a')[2].get('href')

In [ ]:
data_[57].find_all('a')

In [ ]:
playerid[40:]

In [7]:
def nba_draft(year):
    
    """
    Parameters:
    -----------
    
    year: int
        draft year
        
    """
    # format url
    url = 'https://www.basketball-reference.com/draft/NBA_{}.html'.format(year)
    req0 = requests.get(url=url)
    soup = BeautifulSoup(req0.content, 'lxml')

    # find all table rows --> skip first row (headers)
    data_ = soup.find_all('tr')[1:]

    # find draft table 
    player_data = pd.read_html(str(soup.select('table')), match='Player')[0]
    player_data.columns = player_data.columns.droplevel(0)
    
    # remove rows that contain bad data
    player_data = player_data.loc[(player_data['Player'] != 'Player') & 
                                  (player_data['Yrs'] != 'Totals')].reset_index(drop=True)

    # get playerids --> account for different length of 'a' tags
    playerid = []
    for j in data_:
        if len(j.find_all('a')) == 4:
            pid_v0 = j.find_all('a')[2].get('href')
            pid_v1 = pid_v0.replace('players/', '').replace('.html', '').split('/')[2]
            playerid.append(pid_v1)
        elif len(j.find_all('a')) == 3:
            pid_v0 = j.find_all('a')[2].get('href')
            if 'players' in pid_v0:
                pid_v1 = pid_v0.replace('players/', '').replace('.html', '').split('/')[2]
                playerid.append(pid_v1)
            else:
                playerid.append('no_playerid')
        elif len(j.find_all('a')) == 2:
            playerid.append('no_playerid')
    
    # add year and playerids to dataframe
    player_data['draft_year'] = year
    player_data['playerid'] = playerid
    
    # clean columns
    player_data.columns = player_data.columns.str.replace('%', '_pct').str.replace('/', '_').str.\
    replace(' ', '_').str.lower().str.strip()
    
    return player_data.iloc[:, 1:]

In [8]:
A=nba_draft(year=1984)

In [9]:
A.head()

,pk,tm,player,college,yrs,g,mp,pts,trb,ast,...,mp,pts,trb,ast,ws,ws_48,bpm,vorp,draft_year,playerid
0,1,HOU,Hakeem Olajuwon,University of Houston,18,1238,44222,26946,13748,3058,...,35.7,21.8,11.1,2.5,162.8,.177,4.9,77.1,1984,olajuha01
1,2,POR,Sam Bowie,University of Kentucky,10,511,14098,5564,3845,1075,...,27.6,10.9,7.5,2.1,26.9,.091,0.0,7.2,1984,bowiesa01
2,3,CHI,Michael Jordan,University of North Carolina,15,1072,41011,32292,6672,5633,...,38.3,30.1,6.2,5.3,214.0,.250,8.1,104.4,1984,jordami01
3,4,DAL,Sam Perkins,University of North Carolina,17,1286,36598,15324,7666,1975,...,28.5,11.9,6.0,1.5,105.4,.138,1.4,31.4,1984,perkisa01
4,5,PHI,Charles Barkley,Auburn University,16,1073,39330,23757,12546,4215,...,36.7,22.1,11.7,3.9,177.2,.216,7.4,93.5,1984,barklch01
